In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import json
import os
import operator
import random

In [2]:
path = "../StackExchange/Stackoverflow/PostLinks.xml"

In [3]:
postIds = []
relatedPostIds = []
linkTypeId = []

In [4]:
for _, elem in ET.iterparse(path, events=("end",)):
    if elem.tag == "row":
        postIds.append(elem.attrib["PostId"])
        relatedPostIds.append(elem.attrib["RelatedPostId"])
        linkTypeId.append(elem.attrib["LinkTypeId"])
        
    elem.clear()
#df = pd.DataFrame(dict_list)

In [5]:
df = pd.DataFrame()
df["PostId"] = postIds
df["RelatedPostId"] = relatedPostIds
df["LinkTypeId"] = linkTypeId

In [6]:
df.head()

,PostId,RelatedPostId,LinkTypeId
0,109,32412,1
1,1970,617600,1
2,2154,2451138,1
3,2572,209329,1
4,3376,2187,1


In [7]:
relatedPost_dict = {}
cluster_dict = {}
for i, (pid,rid) in enumerate(zip(df["PostId"],df["RelatedPostId"])):
    #duplicate detection
    if rid in relatedPost_dict.keys() and pid in relatedPost_dict[rid]:
        continue

    if pid not in relatedPost_dict.keys():
        relatedPost_dict[pid] = [rid]
    else:
        relatedPost_dict[pid].append(rid)

    if pid not in cluster_dict.keys() and rid not in cluster_dict.keys():
        cluster_dict[pid] = i
        cluster_dict[rid] = i
    elif pid in cluster_dict.keys() and rid not in cluster_dict.keys():
        cluster_dict[rid] = cluster_dict[pid]
    elif pid not in cluster_dict.keys() and rid in cluster_dict.keys():
        cluster_dict[pid] = cluster_dict[rid]

In [8]:
cluster_dict

{'109': 0,
 '32412': 0,
 '1970': 1,
 '617600': 1,
 '2154': 2,
 '2451138': 2,
 '2572': 3,
 '209329': 3,
 '3376': 4,
 '2187': 4,
 '18080': 4,
 '3859': 6,
 '802573': 6,
 '4565': 7,
 '583532': 7,
 '4582': 8,
 '1955663': 8,
 '4850': 9,
 '1369312': 9,
 '4952': 10,
 '1607': 10,
 '173': 10,
 '6371': 10,
 '5724': 13,
 '131955': 13,
 '6173': 14,
 '1732348': 14,
 '7470': 15,
 '82872': 15,
 '8170': 16,
 '245395': 16,
 '9705': 17,
 '305694': 17,
 '9751': 18,
 '371': 18,
 '9926': 19,
 '619677': 19,
 '10199': 20,
 '21280': 20,
 '10293': 21,
 '98915': 21,
 '109762': 21,
 '10524': 23,
 '110314': 23,
 '11028': 24,
 '397089': 24,
 '11330': 25,
 '9410': 25,
 '11743': 26,
 '52176': 26,
 '11919': 27,
 '155308': 27,
 '14279': 28,
 '1711': 28,
 '14413': 29,
 '17370': 29,
 '14810': 30,
 '81543': 30,
 '15240': 31,
 '679979': 31,
 '15266': 32,
 '933331': 32,
 '15732': 33,
 '290326': 33,
 '16161': 34,
 '86515': 34,
 '16167': 35,
 '1209866': 35,
 '16248': 36,
 '56227': 36,
 '16860': 37,
 '1383': 37,
 '17125': 38,


In [9]:
df_st = pd.read_csv("../StackExchange/final_data/stackoverflow_2053988.csv")

C:\Users\wangs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df_st.head()

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ContentLicense,...,ViewCount,Title,Tags,AnswerCount,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate,Keep
0,14137474,1,NaN,2013-01-03T10:55:00.823,4,<p>I am trying to update the probe settings fo...,1383986.0,2013-09-26T14:14:39.440,2,CC BY-SA 3.0,...,2554.0,Update Windows Azure Endpoint with load balanc...,<powershell><azure><load-balancing><endpoint><...,1.0,0.0,NaN,NaN,NaN,NaN,Y
1,14137494,1,NaN,2013-01-03T10:55:57.567,0,<p>I wrote the following test for my PureMVC p...,1765519.0,2013-01-07T16:41:03.967,0,CC BY-SA 3.0,...,130.0,What event is fired when a Mediator is Registe...,<actionscript-3><apache-flex><puremvc><flexunit4>,2.0,NaN,NaN,NaN,NaN,NaN,Y
2,14137606,1,NaN,2013-01-03T11:03:03.710,1,<p>I have a grid that has a JSON date field. I...,75594.0,2016-04-28T06:41:18.827,1,CC BY-SA 3.0,...,1705.0,Formatting dates in koGrid using Moment.Js,<knockout.js><momentjs><kogrid>,1.0,NaN,NaN,NaN,NaN,NaN,Y
3,14137614,1,NaN,2013-01-03T11:03:33.850,1,<p>The original problem i am dealing with is o...,898747.0,2013-01-03T11:29:03.717,0,CC BY-SA 3.0,...,153.0,Does accessing a list change its ref count?,<python><refcounting>,1.0,NaN,NaN,NaN,NaN,NaN,Y
4,14137805,1,NaN,2013-01-03T11:14:14.850,0,<p>I use Tigase as XMPP server. Everything is ...,574475.0,2013-02-04T14:51:05.873,0,CC BY-SA 3.0,...,749.0,Tigase: sometimes i receive offline messages t...,<xmpp><tigase>,1.0,NaN,NaN,NaN,NaN,NaN,Y


In [11]:
len(df_st)

2053988

In [12]:
hasRelatedPost = []

for i in range(len(df_st)):
    pid = df_st.iloc[i]["Id"]
    if str(pid) in relatedPost_dict.keys():
        hasRelatedPost.append("Y")
    else:
        hasRelatedPost.append("N")

In [13]:
df_st["hasRelatedPost"] = hasRelatedPost

In [14]:
cluster_id = []

for i in range(len(df_st)):
    pid = df_st.iloc[i]["Id"]
    if str(pid) in cluster_dict.keys():
        cluster_id.append(cluster_dict[str(pid)])
    else:
        cluster_id.append("N")

In [15]:
len(set(cluster_id))

361009

In [16]:
df_st["cluster"] = cluster_id

## Sentence Cluster

In [17]:
len(df_st[df_st["cluster"] != 'N'])

540593

In [18]:
df_cluster = df_st[df_st["cluster"] != 'N']
df_cluster.reset_index(drop=True, inplace=True)

In [19]:
df_cluster.head()

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ContentLicense,...,Tags,AnswerCount,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate,Keep,hasRelatedPost,cluster
0,14137606,1,NaN,2013-01-03T11:03:03.710,1,<p>I have a grid that has a JSON date field. I...,75594.0,2016-04-28T06:41:18.827,1,CC BY-SA 3.0,...,<knockout.js><momentjs><kogrid>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y,3210680
1,14137614,1,NaN,2013-01-03T11:03:33.850,1,<p>The original problem i am dealing with is o...,898747.0,2013-01-03T11:29:03.717,0,CC BY-SA 3.0,...,<python><refcounting>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y,277741
2,14137989,1,NaN,2013-01-03T11:25:08.090,57,<p>Why doesn't this code throw an <code>Arithm...,1612090.0,2021-05-12T09:23:50.500,9,CC BY-SA 4.0,...,<java><arithmeticexception>,8.0,0.0,NaN,NaN,NaN,NaN,Y,Y,48114
3,14138015,1,NaN,2013-01-03T11:26:26.997,1,<p>I've written a code to run a timer job whic...,1934559.0,2013-01-04T06:03:57.583,0,CC BY-SA 3.0,...,<c#><visual-studio-2010><sharepoint><timer-jobs>,2.0,0.0,NaN,NaN,NaN,NaN,Y,Y,366
4,14138623,1,NaN,2013-01-03T12:05:59.873,15,<p>I am trying to generate a dynamic PDF file ...,1526294.0,2019-10-25T02:58:46.197,1,CC BY-SA 3.0,...,<java><itext><weblogic8.x>,7.0,0.0,NaN,NaN,NaN,NaN,Y,N,2205232


In [20]:
cluster_included = list(df_cluster.groupby(["cluster"])["Id"].count().index.values)
count_list = list(df_cluster.groupby(["cluster"])["Id"].count().values)

In [44]:
cluster_filtered = []
for cluster, count in zip(cluster_included,count_list):
    if count > 9:
        cluster_filtered.append(cluster)

In [45]:
len(cluster_filtered)

1876

In [46]:
df_final = df_cluster[df_cluster["cluster"].isin(cluster_filtered)]

In [47]:
len(df_final)

37254

In [48]:
df_final.reset_index(drop=True, inplace=True)
df_final.head()

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ContentLicense,...,Tags,AnswerCount,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate,Keep,hasRelatedPost,cluster
0,14137614,1,NaN,2013-01-03T11:03:33.850,1,<p>The original problem i am dealing with is o...,898747.0,2013-01-03T11:29:03.717,0,CC BY-SA 3.0,...,<python><refcounting>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y,277741
1,14139111,1,NaN,2013-01-03T12:37:41.767,11,"<p>This question is derived from <a href=""http...",898747.0,2013-01-03T13:59:37.050,3,CC BY-SA 3.0,...,<python><refcounting>,1.0,0.0,NaN,NaN,NaN,NaN,Y,Y,277741
2,14140678,1,NaN,2013-01-03T14:19:03.583,3,<p>I am trying to dig into the GNU Prolog beha...,NaN,2022-08-20T09:24:10.870,2,CC BY-SA 3.0,...,<prolog><dcg><gnu-prolog><iso-prolog>,2.0,NaN,NaN,user502187,user502187,NaN,Y,Y,383325
3,14153667,1,NaN,2013-01-04T08:33:08.067,0,"<p>According to Dave Thomas in <a href=""http:/...",1525842.0,2021-01-08T19:22:16.680,0,CC BY-SA 3.0,...,<ruby><oop><inheritance><metaclass><eigenclass>,2.0,0.0,NaN,NaN,NaN,NaN,Y,Y,230843
4,14175592,1,NaN,2013-01-05T19:38:05.420,1,<p>I'm loading a json file relatively using da...,322902.0,2014-09-01T20:17:08.193,0,CC BY-SA 3.0,...,<intellij-idea><dart><dartium>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y,559666


In [49]:
df_new = pd.DataFrame()
sentence = []
for i in range(len(df_final)):
    title1 = df_final.iloc[i]["Title"]
    body1 = df_final.iloc[i]["Body"]
    s1 = title1.strip() + " " + body1
    sentence.append(s1)
    
df_new["Id"] = df_final["Id"]
df_new["sentence"] = sentence
df_new["tag"] = df_final["Tags"]
df_new["label"] = df_final["cluster"]

In [50]:
df_new.head()

,Id,sentence,tag,label
0,14137614,Does accessing a list change its ref count? <p...,<python><refcounting>,277741
1,14139111,Python disable reference counting for some obj...,<python><refcounting>,277741
2,14140678,Standard way to see what phrase/3 translates? ...,<prolog><dcg><gnu-prolog><iso-prolog>,383325
3,14153667,How does Inheritance work in Ruby? <p>Accordin...,<ruby><oop><inheritance><metaclass><eigenclass>,230843
4,14175592,XMLHttpRequest Cross origin requests In Dart w...,<intellij-idea><dart><dartium>,559666


In [51]:
len(df_new)

37254

In [52]:
df_new.to_csv("../StackExchange/Stackoverflow/cluster2.csv",index=None)

In [4]:
df_new = pd.read_csv("../StackExchange/Stackoverflow/cluster.csv")

In [5]:
len(df_new)

11904

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_new, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)

In [7]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [8]:
print("train:",len(train))
print("val:",len(val))
print("test:",len(test))

train: 9641
val: 1072
test: 1191


In [9]:
train.head()

,Id,sentence,tag,label
0,610245,"Where and why do I have to put the ""template"" ...",<c++><templates><typename><c++-faq><dependent-...,144924
1,10761867,Save android activity state <p>I try to create...,<android><state><android-activity><activity-li...,34578
2,10231118,"Change JProgressBar value, while Thread is run...",<java><multithreading><swing><user-interface><...,246881
3,63096672,difference between passing parameters in funct...,<c++><recursion><parameter-passing><pass-by-re...,4146
4,59141326,How to display upcoming birthdays with java in...,<java><android><android-date><date>,1166157


In [10]:
train.to_csv("../StackExchange/Stackoverflow/cluster/train1.csv",index = None)
val.to_csv("../StackExchange/Stackoverflow/cluster/val1.csv",index = None)
test.to_csv("../StackExchange/Stackoverflow/cluster/test1.csv",index = None)

## Sentence Pair

In [13]:
len(df_st[df_st["hasRelatedPost"] == 'Y'])

339312

In [19]:
df_sample = df_st[df_st["hasRelatedPost"] == 'Y']
df_sample.reset_index(drop=True)

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ContentLicense,...,Title,Tags,AnswerCount,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate,Keep,hasRelatedPost
0,14137606,1,NaN,2013-01-03T11:03:03.710,1,<p>I have a grid that has a JSON date field. I...,75594.0,2016-04-28T06:41:18.827,1,CC BY-SA 3.0,...,Formatting dates in koGrid using Moment.Js,<knockout.js><momentjs><kogrid>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y
1,14137614,1,NaN,2013-01-03T11:03:33.850,1,<p>The original problem i am dealing with is o...,898747.0,2013-01-03T11:29:03.717,0,CC BY-SA 3.0,...,Does accessing a list change its ref count?,<python><refcounting>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y
2,14137989,1,NaN,2013-01-03T11:25:08.090,57,<p>Why doesn't this code throw an <code>Arithm...,1612090.0,2021-05-12T09:23:50.500,9,CC BY-SA 4.0,...,Java division by zero doesnt throw an Arithmet...,<java><arithmeticexception>,8.0,0.0,NaN,NaN,NaN,NaN,Y,Y
3,14138015,1,NaN,2013-01-03T11:26:26.997,1,<p>I've written a code to run a timer job whic...,1934559.0,2013-01-04T06:03:57.583,0,CC BY-SA 3.0,...,How to make a timer job run only once?,<c#><visual-studio-2010><sharepoint><timer-jobs>,2.0,0.0,NaN,NaN,NaN,NaN,Y,Y
4,14139058,1,NaN,2013-01-03T12:34:07.967,1,<p>I have a C++ application and I would like t...,1849534.0,2013-01-10T09:38:30.740,2,CC BY-SA 3.0,...,autobuild of Lua api from C++ application,<c++><api><lua><auto-build>,2.0,0.0,NaN,NaN,NaN,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339307,14137162,1,NaN,2013-01-03T10:38:09.050,0,"<p>Used <a href=""https://stackoverflow.com/que...",1484000.0,2013-01-03T14:06:11.250,0,CC BY-SA 3.0,...,AJax Callback fails (NPE) on selectoneitem,<ajax><jsf-2><nullpointerexception><jboss7.x>,2.0,NaN,NaN,NaN,NaN,NaN,Y,Y
339308,14137208,1,NaN,2013-01-03T10:40:41.080,1,<p>I am designing a <em>paying</em> app that w...,1503888.0,2013-11-23T21:08:22.290,0,CC BY-SA 3.0,...,Paid App : need to uniquely ID the user,<iphone><uniqueidentifier>,1.0,NaN,NaN,NaN,NaN,NaN,Y,Y
339309,14137273,1,NaN,2013-01-03T10:43:58.103,13,<p>I'm looking to query a database of wine nam...,1354771.0,2019-02-02T16:17:51.730,1,CC BY-SA 3.0,...,MySQL REGEXP query - accent insensitive search,<mysql><regex><diacritics><accent-insensitive>,7.0,0.0,NaN,NaN,NaN,NaN,Y,Y
339310,14137301,1,NaN,2013-01-03T10:45:44.093,0,<p>I was wondering what are the demerits of us...,1412255.0,2013-01-03T11:38:58.583,3,CC BY-SA 3.0,...,What are the demerits of using XML over HTML?,<html><xml><webpage><data-presentation>,2.0,NaN,2013-01-03T15:44:13.430,NaN,NaN,NaN,Y,Y


In [15]:
index_dict = {}

for index in range(len(df_sample)):
    pid = df_sample.iloc[index]["Id"]
    index_dict[pid] = index


In [16]:
len(index_dict)

339312

In [20]:
sentence1 = []
sentence2 = []
df_tmp = pd.DataFrame()

for i in range(280000,len(df_sample),1):
    pid = df_sample.iloc[i]["Id"]
    title1 = df_sample.iloc[i]["Title"]
    body1 = df_sample.iloc[i]["Body"]
    s1 = title1.strip() + " " + body1
    
    for rid in relatedPost_dict[str(pid)]:
        if int(rid) in list(df_sample["Id"].values):
            title2 = df_sample.iloc[index_dict[int(rid)]]["Title"]
            body2 = df_sample.iloc[index_dict[int(rid)]]["Body"]
            s2 = title2.strip() + " " + body2
            
            sentence1.append(s1)
            sentence2.append(s2)
    
    
    
    
    if len(sentence1) != 0 and len(sentence1) % 2000 == 0:
        #df_tmp = pd.DataFrame()
        df_tmp["sentence1"] = sentence1
        df_tmp["sentence2"] = sentence2
        df_tmp.to_csv("../StackExchange/Stackoverflow/data_sentence_pairs3.csv",index = None)
        #break
    

KeyboardInterrupt: 

In [ ]:
df_sample = df_sample.sample(frac=0.3, random_state=1)

In [57]:
len(df_sample)

103907

In [63]:
len(rPostHere)

95339

In [3]:
df = pd.read_csv("../StackExchange/Stackoverflow/data_sentence_pairs.csv")

In [4]:
len(df)

39744

In [5]:
df.head()

,sentence1,sentence2
0,Does accessing a list change its ref count? <p...,Shared memory in multiprocessing <p>I have thr...
1,Python disable reference counting for some obj...,Shared memory in multiprocessing <p>I have thr...
2,Standard way to see what phrase/3 translates? ...,"What is the difference between ' and "" in Prol..."
3,CurrentUICulture ignores Region and Language s...,"Use ""real"" CultureInfo.CurrentCulture in WPF B..."
4,How does Inheritance work in Ruby? <p>Accordin...,Why are symbols in Ruby not thought of as a ty...


In [26]:
df_sample_val = df.sample(frac=0.12, random_state=1)

index_list = [i for i in range(0,len(df))]
index_train = set(index_list).difference(set(df_sample_val.index.values))
df_train = df.loc[list(index_train)]

print("#validation:",len(df_sample_val))
print("#training:",len(df_train))

#validation: 4769
#training: 34975


## Generating negative samples

In [17]:
df_random = df_st[df_st["hasRelatedPost"] == 'N']
df_random.reset_index(drop=True)

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ContentLicense,...,Title,Tags,AnswerCount,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate,Keep,hasRelatedPost
0,14137474,1,NaN,2013-01-03T10:55:00.823,4,<p>I am trying to update the probe settings fo...,1383986.0,2013-09-26T14:14:39.440,2,CC BY-SA 3.0,...,Update Windows Azure Endpoint with load balanc...,<powershell><azure><load-balancing><endpoint><...,1.0,0.0,NaN,NaN,NaN,NaN,Y,N
1,14137494,1,NaN,2013-01-03T10:55:57.567,0,<p>I wrote the following test for my PureMVC p...,1765519.0,2013-01-07T16:41:03.967,0,CC BY-SA 3.0,...,What event is fired when a Mediator is Registe...,<actionscript-3><apache-flex><puremvc><flexunit4>,2.0,NaN,NaN,NaN,NaN,NaN,Y,N
2,14137805,1,NaN,2013-01-03T11:14:14.850,0,<p>I use Tigase as XMPP server. Everything is ...,574475.0,2013-02-04T14:51:05.873,0,CC BY-SA 3.0,...,Tigase: sometimes i receive offline messages t...,<xmpp><tigase>,1.0,NaN,NaN,NaN,NaN,NaN,Y,N
3,14137819,1,NaN,2013-01-03T11:15:00.433,2,<p>When I open my installer(that I created usi...,1945221.0,2013-02-18T18:55:07.787,0,CC BY-SA 3.0,...,Setting the UAC 'Publisher' Field for a Instal...,<java><windows><installanywhere>,1.0,0.0,NaN,NaN,NaN,NaN,Y,N
4,14137894,1,NaN,2013-01-03T11:19:14.627,2,<p>My vcards have a CATEGORIES property that h...,470914.0,2013-01-03T14:50:08.683,0,CC BY-SA 3.0,...,How to do advanced filters in CardDAV (WebDAV)...,<http><filter><report><webdav><carddav>,1.0,0.0,NaN,NaN,NaN,NaN,Y,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714671,14137291,1,NaN,2013-01-03T10:45:05.453,3,"<p>I believe the title is self-explanatory, bu...",298054.0,2019-04-17T19:30:50.540,2,CC BY-SA 4.0,...,How can a macro define a valid global name bas...,<c><generics><macros><c11>,2.0,NaN,NaN,NaN,NaN,NaN,Y,N
1714672,14137313,1,NaN,2013-01-03T10:46:32.257,1,<p>Hi i am creating a pdf with texts and image...,552712.0,2013-02-21T01:23:49.353,0,CC BY-SA 3.0,...,Restrict the content of pdf to single page cre...,<android><pdf><pdf-generation><android-droidtext>,1.0,NaN,NaN,NaN,NaN,NaN,Y,N
1714673,14137318,1,NaN,2013-01-03T10:46:35.963,2,<p>From config/main.php :</p>\n\n<pre><code> ...,785309.0,2013-01-03T12:05:12.620,0,CC BY-SA 3.0,...,Gii not detecting mysql tables in model creator,<yii><gii>,1.0,NaN,NaN,NaN,NaN,NaN,Y,N
1714674,14137321,1,NaN,2013-01-03T10:46:52.250,2,<p>i want design my navigationBar like followi...,1684977.0,2013-01-17T07:06:34.557,1,CC BY-SA 3.0,...,how do i achieve this on navigationBar,<iphone><uinavigationcontroller><uibutton><uin...,3.0,0.0,NaN,NaN,NaN,NaN,Y,N


In [18]:
sentence1 = []
sentence2 = []
df_tmp = pd.DataFrame()

while len(sentence1) <= 8000:
    i1 = random.randrange(0, len(df_random), 1)
    i2 = random.randrange(0, len(df_random), 1)
    while i2 == i1:
        i2 = random.randrange(0, len(df_random), 1)
    
    
    title1 = df_random.iloc[i1]["Title"]
    body1 = df_random.iloc[i1]["Body"]
    s1 = title1.strip() + " " + body1
    

    title2 = df_random.iloc[i2]["Title"]
    body2 = df_random.iloc[i2]["Body"]
    s2 = title2.strip() + " " + body2
            
    sentence1.append(s1)
    sentence2.append(s2)
    
df_tmp["sentence1"] = sentence1
df_tmp["sentence2"] = sentence2
df_tmp.to_csv("../StackExchange/Stackoverflow/data_not_pairs3.csv",index = None)

## Training data

In [21]:
df_p1 = pd.read_csv("../StackExchange/Stackoverflow/data_sentence_pairs.csv",encoding='utf-8')
df_p2 = pd.read_csv("../StackExchange/Stackoverflow/data_sentence_pairs2.csv",encoding='utf-8')
df_p3 = pd.read_csv("../StackExchange/Stackoverflow/data_sentence_pairs3.csv",encoding='utf-8')
df_np1 = pd.read_csv("../StackExchange/Stackoverflow/data_not_pairs.csv",encoding='iso-8859-1')
df_np2 = pd.read_csv("../StackExchange/Stackoverflow/data_not_pairs2.csv",encoding='utf-8')
df_np3 = pd.read_csv("../StackExchange/Stackoverflow/data_not_pairs3.csv",encoding='utf-8')

In [22]:
df_p1["label"] = [1 for i in range(len(df_p1))]
df_p2["label"] = [1 for i in range(len(df_p2))]
df_p3["label"] = [1 for i in range(len(df_p3))]
df_np1["label"] = [0 for i in range(len(df_np1))]
df_np2["label"] = [0 for i in range(len(df_np2))]
df_np3["label"] = [0 for i in range(len(df_np3))]

print("df_p1:",len(df_p1))
print("df_p2:",len(df_p2))
print("df_p3:",len(df_p3))
print("df_np1:",len(df_np1))
print("df_np2:",len(df_np2))
print("df_np3:",len(df_np3))

df_p1: 39744
df_p2: 5000
df_p3: 1999
df_np1: 44981
df_np2: 9242
df_np3: 5740


In [23]:
df_train = pd.concat([df_p1,df_np1])
df_test = pd.concat([df_p2,df_np2])
df_val = pd.concat([df_p3,df_np3])

print("training data:",len(df_train))
print("validation data:",len(df_val))
print("testing data:",len(df_test))

training data: 84725
validation data: 7739
testing data: 14242


In [76]:
df_train.head()

,sentence1,sentence2,label
0,Does accessing a list change its ref count? <p...,Shared memory in multiprocessing <p>I have thr...,1
1,Python disable reference counting for some obj...,Shared memory in multiprocessing <p>I have thr...,1
2,Standard way to see what phrase/3 translates? ...,"What is the difference between ' and "" in Prol...",1
3,CurrentUICulture ignores Region and Language s...,"Use ""real"" CultureInfo.CurrentCulture in WPF B...",1
4,How does Inheritance work in Ruby? <p>Accordin...,Why are symbols in Ruby not thought of as a ty...,1


In [78]:
df_train.to_csv("../StackExchange/Stackoverflow/train.csv",index = None)
df_val.to_csv("../StackExchange/Stackoverflow/val.csv",index = None)
df_test.to_csv("../StackExchange/Stackoverflow/test.csv",index = None)